### Configuración Inicial

In [1]:
!pip uninstall -y numpy
!pip install numpy==1.26

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 43.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.


In [1]:
!pip install scikit-surprise --no-build-isolation --no-deps
!pip install memory_profiler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2708514 sha256=ae67b36b579d63d796bdf095873ff1b91cdd04d122839d5452a1f3f165ae8d4b
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


### Instalación de Librerías

In [2]:
import pandas as pd
import gdown
from surprise import Dataset, Reader, SVDpp, accuracy
from surprise.model_selection import train_test_split
import time
from memory_profiler import memory_usage
from collections import defaultdict

### Importación de los Datos

In [3]:
gdown.download(id='1H_24ycns6zbOVfHFJRI9vGjVffVA5z6v', output='training_ratings.csv', quiet=False)
gdown.download(id='1pKmf07ehHOmlvIyT8nv__vPuWE2Z3ygZ', output='validation_ratings.csv', quiet=False)

df_train = pd.read_csv('training_ratings.csv')
df_val = pd.read_csv('validation_ratings.csv')

Downloading...
From (original): https://drive.google.com/uc?id=1H_24ycns6zbOVfHFJRI9vGjVffVA5z6v
From (redirected): https://drive.google.com/uc?id=1H_24ycns6zbOVfHFJRI9vGjVffVA5z6v&confirm=t&uuid=2d41e57f-6485-42e8-832b-08927155d9c7
To: /content/training_ratings.csv
100%|██████████| 249M/249M [00:02<00:00, 87.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1pKmf07ehHOmlvIyT8nv__vPuWE2Z3ygZ
To: /content/validation_ratings.csv
100%|██████████| 58.3M/58.3M [00:00<00:00, 71.5MB/s]


### Preprocesamiento de Datos

In [4]:
df_train.drop_duplicates(inplace=True, subset=['user', 'item'])
df_val.drop_duplicates(inplace=True, subset=['user', 'item'])

In [9]:
print(f"Tamaño original del training set: {len(df_train)}")

# se obtiene un sample debido a que hay muchos datos y se demora mucho
df_train_sample = df_train.sample(n=1000000, random_state=42)
print(f"Tamaño del nuevo training set (muestra): {len(df_train_sample)}")


Tamaño original del training set: 12390406
Tamaño del nuevo training set (muestra): 1000000


### Configuración de Experimentos

In [10]:
reader = Reader(rating_scale=(0, 10))
train_data = Dataset.load_from_df(df_train_sample[['user', 'item', 'rating']], reader)
trainset = train_data.build_full_trainset()

validation_tuples = [tuple(x) for x in df_val[['user', 'item', 'rating']].to_numpy()]

### Predicción de ratings y top N



In [15]:
def evaluar_svdpp(trainset, testset, n_factors, n_epochs, lr_all, reg_all, verbose):
    """
    Entrena, predice y evalúa el modelo SVD++, retornando el RMSE y MAE.
    """
    algo = SVDpp(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_all=reg_all, verbose=verbose)

    algo.fit(trainset)

    predictions = algo.test(testset)

    rmse = accuracy.rmse(predictions)
    mae = accuracy.mae(predictions)

    return rmse, mae, predictions

start_time = time.time()

rmse_resultado, mae_resultado, predictions = evaluar_svdpp(trainset, validation_tuples, 50, 20, 0.01, 0.02, True)

end_time = time.time()
elapsed_time = end_time - start_time

memoria_usada = memory_usage((evaluar_svdpp, (trainset, validation_tuples, 50, 20, 0.01, 0.02, False)))
memoria_max = max(memoria_usada) - min(memoria_usada)

print(f"\n--- Resumen de Rendimiento (SVD++) ---")
print(f"RMSE final: {rmse_resultado:.4f}")
print(f"MAE final: {mae_resultado:.4f}")
print(f"Tiempo de ejecución: {elapsed_time:.2f} segundos")
print(f"Memoria utilizada: {memoria_max:.2f} MB")

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
RMSE: 1.3123
MAE:  0.9971
RMSE: 1.3119
MAE:  0.9970

--- Resumen de Rendimiento (SVD++) ---
RMSE final: 1.3123
MAE final: 0.9971
Tiempo de ejecución: 195.43 segundos
Memoria utilizada (pico): 862.64 MB


In [14]:
def get_top_n(predictions, n=10):
    """Devuelve las N-mejores recomendaciones para cada usuario."""
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n_predictions = get_top_n(predictions, n=10)

user_id_ejemplo = df_val['user'].iloc[0]
if user_id_ejemplo in top_n_predictions:
    print(f"\nTop 10 predicciones para el usuario '{user_id_ejemplo}':")
    for iid, est_rating in top_n_predictions[user_id_ejemplo]:
        print(f"  Ítem ID: {iid}, Rating Predicho: {est_rating:.2f}")


Top 10 predicciones para el usuario ' beastvol':
  Ítem ID: 18602, Rating Predicho: 7.52
